In [ ]:
%matplotlib inline
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
from torch.autograd import Variable
import nibabel as nib
import pandas as pd
from torch.utils import data
import random
from sklearn.model_selection import KFold
import argparse
import os
import time,statsmodels
import sys
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit
from scipy.spatial import distance_matrix
import pathlib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,balanced_accuracy_score,precision_score,f1_score,silhouette_score,recall_score,classification_report,confusion_matrix,roc_curve
from scipy.stats import ttest_ind
from mpl_toolkits.mplot3d import Axes3D   
from sklearn.manifold import TSNE
import plotly.express as px
from sklearn.decomposition import PCA
import seaborn as sns
from IPython.display import display, HTML
from scipy.stats import spearmanr,pearsonr, ttest_ind
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.linear_model import LinearRegression

display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '#notebook-container {padding: 0px;}'
    '</style>'
))

# Get ACC

In [ ]:
# Harmed_2DSminmax, ds then minmax

data_name = "T1_ADVDLBD_Mlabel_10fold_NoHarm"
n_epochs, learning_rate, ds = 300, '0.0001', '3DS'
norm = 'minmax'

which_model = "resnet18_64_linear"

highest = 0
all_acc = []
full_model = []
all_pred = []
all_target = []
best_pth = 0
b_size = 8

for b_size in [8]:

    loss_name = 'ASL_gn4_gp1_c0.1'
#     loss_name = 'BCE'
#     sigma = '0.1'
#     model_path = "/home/dw/Desktop/DemCLF/model//results/task2_" + data_name + "_" +ds + norm+ "_OTF/" + which_model+ "_mlr" + str(learning_rate) + \
#     "_ep" + str(n_epochs)  + '_batch' + str(b_size)+ "_"+ loss_name+ "_sigma" + sigma+"/"
    
    model_path = "/media/dw/Data/BINC_T1/BINC/dropbox/DW/results_GenieCode/task2_T1_ADVDLBD_Mlabel_10fold_0307v2_3DSminmax_OTF//" +\
    which_model+ "_mlr0.0001_ep300_batch" + str(b_size)+ "_"+ loss_name+ "_sigma0.1_wd0.001/"
    

    
 
    all_t = []
    all_p = []
    for fold in range(0,10):   
        save_name = 'fold'+str(fold)  
        y_target = np.load(model_path + save_name +  "_target.npy")
        y_predict = np.load(model_path + save_name + "_predict.npy")
        all_t.extend(y_target)
        all_p.extend(y_predict)
        
    all_p = np.array(all_p).astype(int)
    all_t = np.array(all_t).astype(int)
    
#     all_p2 = convert_onehot(all_p)
#     all_t2 = convert_onehot(all_t)
    
    
    ac1 = precision_score(all_t, all_p, average='micro',zero_division =0)
    ac2 = recall_score(all_t, all_p, average='micro')
    ac3 = f1_score(all_t, all_p, average='micro')
    ac4 = balanced_accuracy_score(all_t[:,0],all_p[:,0])
    ac5 = balanced_accuracy_score(all_t[:,1],all_p[:,1])
    ac6 = balanced_accuracy_score(all_t[:,2],all_p[:,2])
    ac7 = (ac4 + ac5 + ac6)/3
#     ac7 = balanced_accuracy_score(all_t2,all_p2)*100
#     n0,n1,n2,n3,av = get_correct(all_p2,all_t2)
        
    t = 3
    print("\n #### batch = ",b_size)
    print('precision',np.round(ac1,t))
    print('recall',np.round(ac2,t))
    print('F1',np.round(ac3,t))
    print('AD acc',np.round(ac4,t))
    print('VD acc',np.round(ac5,t))
    print('LBD acc',np.round(ac6,t))
    print('AVE acc',np.round(ac7,t))

    
    if np.round(np.mean(ac3),2)> highest:
        highest = np.round(np.mean(ac3),2)
        best_pth = model_path
    
print("\nhighest:",highest) 
print(best_pth)

al_l = []

for fold in range(0,10):
    df = pd.read_csv(best_pth + "fold" + str(fold) + "_prob.csv")
    df = df[(df.tp == 'test')]
    df =df[['mri_ID','probAD', 'probVD', 'probLBD']]
    df['mri_ID'] = df['mri_ID'].str[:19]
    al_l.extend(df.values.tolist())
df = pd.DataFrame(al_l, columns = ['mri_ID','probAD', 'probVD', 'probLBD'])
# a = pd.read_csv("/home/dw/Desktop/Genie/demCLF_kfold_T1/data_p2/allADVDLBD.csv")
a = pd.read_csv("/home/dw/Desktop/DemCLF/data/allADVDLBD_0209.csv")
df = pd.merge(df,a,on='mri_ID',how='left')
    

target_names = ['AD', 'VD','LBD']
print(classification_report(all_t, all_p, target_names=target_names))

# Chart

In [ ]:
fig,axs = plt.subplots(ncols = 3,nrows=1,figsize = (15,5))
axs = axs.flatten()


s = 150    
f1,f2 ='probAD','probVD'

fs = 20

# axs[0].scatter(-100,-100, label = "CN", marker='o', alpha = 0.5, s =40, color = 'gray')
# axs[0].scatter(-100,-100, label = "AD", marker='>', alpha = 0.5, s =s, color = '#2CA02C')
# axs[0].scatter(-100,-100, label = "VD", marker='<', alpha = 0.5, s =s, color = '#D62728')
# axs[0].scatter(-100,-100, label = "LBD", marker='^', alpha = 0.5, s =s, color = 'blue')
# axs[0].scatter(-100,-100, label = "AD+VD", marker='+', alpha = 0.5, s =s+100, color = 'violet')
# axs[0].scatter(-100,-100, label = "AD+LBD", marker='+', alpha = 0.5, s =s+100, color = 'purple')
# axs[0].scatter(-100,-100, label = "VD+LBD", marker='+', alpha = 0.5, s =s+100, color = 'magenta')
# axs[0].scatter(-100,-100, label = "AD+VD+LBD", marker='x', alpha = 0.5, s =s, color = 'goldenrod')
# axs[0].legend(prop={'size': 20},bbox_to_anchor=(1, 0.53, 0.5, 0.5))

axs[0].scatter(df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 0)][[f1]],
            df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 0)][[f2]], 
             marker='o', alpha = 0.3, s =40, color = 'gray')

axs[0].scatter(df.loc[(df['AD'] == 1) &(df['VD'] == 1)&(df['LBD'] == 0)][[f1]],
            df.loc[(df['AD'] == 1) &(df['VD'] == 1)&(df['LBD'] == 0)][[f2]], 
             marker='+', alpha = 0.5, s =s+100, color = 'violet')

axs[0].scatter(df.loc[(df['AD'] == 1) &(df['VD'] == 1)&(df['LBD'] == 1)][[f1]],
            df.loc[(df['AD'] == 1) &(df['VD'] == 1)&(df['LBD'] == 1)][[f2]], 
            marker='x', alpha = 0.5, s =s, color = 'goldenrod')
axs[0].scatter(df.loc[(df['AD'] == 0) &(df['VD'] == 1)&(df['LBD'] == 0)][[f1]],
            df.loc[(df['AD'] == 0) &(df['VD'] == 1)&(df['LBD'] == 0)][[f2]], 
             marker='<', alpha = 0.5, s =s, color = '#D62728')

axs[0].scatter(df.loc[(df['AD'] == 1) &(df['VD'] == 0)&(df['LBD'] == 0)][[f1]],
            df.loc[(df['AD'] == 1) &(df['VD'] == 0)&(df['LBD'] == 0)][[f2]], 
             marker='>', alpha = 0.5, s =s, color = '#2CA02C')

axs[0].set_xlabel(r'$\bf{AD}$ Probability', fontsize = fs)
axs[0].set_ylabel(r'$\bf{VD}$ Probability', fontsize = fs)    



f1,f2 ='probAD','probLBD'
axs[1].scatter(df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 0)][[f1]],
            df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 0)][[f2]], 
            marker='o', alpha = 0.3, s =40, color = 'gray')

axs[1].scatter(df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 1)][[f1]],
            df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 1)][[f2]], 
             marker='^', alpha = 0.5, s =s, color = 'blue')

axs[1].scatter(df.loc[(df['AD'] == 1) &(df['VD'] == 0)&(df['LBD'] == 0)][[f1]],
            df.loc[(df['AD'] == 1) &(df['VD'] == 0)&(df['LBD'] == 0)][[f2]], 
            marker='>', alpha = 0.5, s =s, color = 'green')

axs[1].scatter(df.loc[(df['AD'] == 1) &(df['VD'] == 0)&(df['LBD'] == 1)][[f1]],
            df.loc[(df['AD'] == 1) &(df['VD'] == 0)&(df['LBD'] == 1)][[f2]], 
             marker='+', alpha = 0.5, s =s+100, color = 'purple')

axs[1].scatter(df.loc[(df['AD'] == 1) &(df['VD'] == 1)&(df['LBD'] == 1)][[f1]],
            df.loc[(df['AD'] == 1) &(df['VD'] == 1)&(df['LBD'] == 1)][[f2]], 
             marker='x', alpha = 0.5, s =s, color = 'goldenrod')

axs[1].set_xlabel(r'$\bf{AD}$ Probability', fontsize = fs)
axs[1].set_ylabel(r'$\bf{LBD}$ Probability', fontsize = fs) 


f1,f2 ='probVD','probLBD'
axs[2].scatter(df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 0)][[f1]],
            df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 0)][[f2]], 
            marker='o', alpha = 0.3, s =40, color = 'gray')

axs[2].scatter(df.loc[(df['AD'] == 0) &(df['VD'] == 1)&(df['LBD'] == 0)][[f1]],
            df.loc[(df['AD'] == 0) &(df['VD'] == 1)&(df['LBD'] == 0)][[f2]], 
            marker='<', alpha = 0.5, s =s, color = '#D62728')

axs[2].scatter(df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 1)][[f1]],
            df.loc[(df['AD'] == 0) &(df['VD'] == 0)&(df['LBD'] == 1)][[f2]], 
            marker='^', alpha = 0.5, s =s, color = 'blue')

axs[2].scatter(df.loc[(df['AD'] == 0) &(df['VD'] == 1)&(df['LBD'] == 1)][[f1]],
            df.loc[(df['AD'] == 0) &(df['VD'] == 1)&(df['LBD'] == 1)][[f2]], 
             marker='+', alpha = 0.5, s =s+100, color = 'magenta')

axs[2].scatter(df.loc[(df['AD'] == 1) &(df['VD'] == 1)&(df['LBD'] == 1)][[f1]],
            df.loc[(df['AD'] == 1) &(df['VD'] == 1)&(df['LBD'] == 1)][[f2]], 
             marker='x', alpha = 0.5, s =s, color = 'goldenrod')

axs[2].set_xlabel( r'$\bf{VD}$ Probability', fontsize = fs)
axs[2].set_ylabel(r'$\bf{LBD}$ Probability', fontsize = fs)
    


# plt.xlim([-0.05,1.05])
# plt.ylim([-0.05,1.05])
# plt.gca().set_yticklabels([f'{x:.0%}' for x in plt.gca().get_yticks()]) 
# plt.gca().set_xticklabels([f'{x:.0%}' for x in plt.gca().get_xticks()]) 

# axs[0].set_title("Deep Learning Chart", fontsize = 20)
# plt.grid(linestyle = '--')
for i in range(0,3):
    axs[i].axvline(x = 0.5, linestyle = '--', color='gray', alpha = 0.5)
    axs[i].axhline(y = 0.5, linestyle = '--', color='gray', alpha = 0.5)
    axs[i].set_xlim([-0.05,1.05])
    axs[i].set_ylim([-0.05,1.05])
    
for i in range(0,3):    
    axs[i].set_yticklabels([f'{x:.0%}' for x in plt.gca().get_yticks()], fontsize=12) 
    axs[i].set_xticklabels([f'{x:.0%}' for x in plt.gca().get_xticks()], fontsize=12) 

    
fig.tight_layout()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(df.AD.values.tolist(), df.probAD.values.tolist())
roc_auc = metrics.auc(fpr, tpr)

fpr2, tpr2, threshold2 = metrics.roc_curve(df.VD.values.tolist(), df.probVD.values.tolist())
roc_auc2 = metrics.auc(fpr2, tpr2)

fpr3, tpr3, threshold3 = metrics.roc_curve(df.LBD.values.tolist(), df.probLBD.values.tolist())
roc_auc3 = metrics.auc(fpr3, tpr3)

plt.figure(figsize=(6, 6))
plt.title('Deep Learning', fontsize = 20)
plt.plot(fpr, tpr, 'g', label = 'AD AUC = %0.2f' % roc_auc)
plt.plot(fpr2, tpr2, 'r', label = 'VD AUC = %0.2f' % roc_auc2)
plt.plot(fpr3, tpr3, 'b', label = 'LBD AUC = %0.2f' % roc_auc3)

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'y--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
plt.grid(linestyle = '--')
plt.ylabel('True Positive Rate', fontsize = 15)
plt.xlabel('False Positive Rate', fontsize = 15)
plt.show()

# DeepSPARE Validation

In [ ]:
pt = 0.05
def ln(name, X, y):
    X2 = sm.add_constant(X)
    est = sm.RLM(y, X2, M=sm.robust.norms.HuberT())
    est2 = est.fit()
    
    s1 = est2.summary2(float_format='%.2f')
    s2 = est2.summary2().tables[1]
    return s2.iloc[0][0], s2.iloc[1][0],s1.tables[1].iloc[1,3]




def get_r2(X,y):
    model = LinearRegression()
    model.fit(X, y)
    return round(model.score(X, y),2)

# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    n1, n2 = len(d1), len(d2)
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    u1, u2 = np.mean(d1), np.mean(d2)
    return (u1 - u2) / s


def get_test(dfi, score):
    ##############################################
    ################### cognitive ################
    ##############################################
    
    if score == 'MMSE':  # lower = severe
        c = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv")[["mri_ID", 'NACCMMSE']].rename(columns = {'NACCMMSE' : score})[["mri_ID", score]]
        d = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_lrp/ADNI_Master_SpreadSheet.csv").rename(columns ={'ID_Date': 'mri_ID'})[["mri_ID", score]]
        b = pd.concat([c,d], axis = 0)
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(float)
        min_ = 1
        max_ = 30

    if score =='CDRGLOB': # higher = severe
        c = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv")[["mri_ID", score]]
        d = pd.read_csv("/home/dw/Desktop/ADNI/Neuropsychological/histo_CDR.csv")[["mri_ID", 'CDGLOBAL']].rename(columns = {'CDGLOBAL' : score})
        b = pd.concat([c,d], axis = 0)
        min_ = 0
        max_ = 3
    
    if score == 'BOSTON': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv", low_memory=False)[["mri_ID", score]]
        min_ = 0
        max_ = 30

    if score == 'TRAILB': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv", low_memory=False)[["mri_ID", score]]
        min_ = 0
        max_ = 299
        
    if score == 'LOGIMEM': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv", low_memory=False)[["mri_ID", score]]
        min_ = 0
        max_ = 26
    if score == 'CRAFTDVR': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv", low_memory=False)[["mri_ID", score]]
        min_ = 0
        max_ = 44
        

        
    ##############################################
    ################### neuropath ################
    ##############################################

    if score == 'NPAMY':  # higher = severe Cerebral amyloid angiopathy
        c = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv", low_memory=False)[["mri_ID", 'NACCAMY']].rename(columns = {'NACCAMY' : score})
        d = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_autopsy/ADNI_KL_0815.csv")[["mri_ID", score]]
        b = pd.concat([c,d], axis = 0)
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(int)
        b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 3
        
    if score == 'NPINF':  # Infarcts and lacunes
        c = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv", low_memory=False)[["mri_ID", 'NACCINF']].rename(columns = {'NACCINF' : score})
        d = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_autopsy/ADNI_KL_0815.csv")[["mri_ID", score]]
        b = pd.concat([c,d], axis = 0)
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(int)
        b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 2
        
    if score == 'NACCARTE':  # Arteriolosclerosis
        b = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv", low_memory=False)[["mri_ID", score]]
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(int)
        b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 3
        
    if score == 'NPATGSEV':  # Arteriolosclerosis
        b = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv", low_memory=False)[["mri_ID", score]]
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(int)
        b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 3
        
    if score == 'NPNLOSS':  # Arteriolosclerosis
        b = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv", low_memory=False)[["mri_ID", score]]
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(int)
        b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 3

    if score == 'NPTHAL':  #Thal phase for amyloid plaques
        c = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv")[["mri_ID", score]]
        d = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_autopsy/ADNI_KL_0815.csv")[["mri_ID", score]]
        b = pd.concat([c,d], axis = 0)
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(int)
        b[score] = np.where(b[score] == 2, 1, b[score])
        b[score] = np.where(b[score] == 3, 2, b[score])
        b[score] = np.where(b[score] == 4, 3, b[score])
        b[score] = np.where(b[score] == 5, 3, b[score])
        b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 5

    if score == 'NPBRAAK': #Braak stage for neurofibrillary degeneration
        c = pd.read_csv('/home/dw/Desktop/DemCLF/label_0504/NACC_Database_Labels_05_04_23.csv')[["mri_ID", 'NACCBRAA']].rename(columns = {'NACCBRAA':score})
        d = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_autopsy/ADNI_KL_0815.csv")[["mri_ID", score]] 
        b = pd.concat([c,d], axis = 0)
        b = b[b[score] == b[score]]
        b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 6

    if score == 'NPNEUR':  # NACCNEUR: Density of neocortical neuritic plaques 3 = Frequent neuritic plaques (C3) 0 = No neuritic plaques (C0)
        c = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv")[["mri_ID", 'NACCNEUR']].rename(columns = {'NACCNEUR':score})
        d = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_autopsy/ADNI_KL_0815.csv")[["mri_ID", score]] 
        b = pd.concat([c,d], axis = 0)
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(int)
        b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 3
        
    if score == 'NACCLEWY': #Lewy body pathology changed
        b = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv")[["mri_ID", score]]
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(int)
#         b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 3
        
    if score == 'NPLBOD': #Lewy body pathology changed
        c = pd.read_csv("/home/dw/Desktop/NACC/mri_ID_w_diag.csv")[["mri_ID", score]]
        d = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_autopsy/ADNI_KL_0815.csv")[["mri_ID", score]] 
        b = pd.concat([c,d], axis = 0)
        b = b[b[score] == b[score]]
        b[score] = b[score].astype(int)
        b.mri_ID = b.mri_ID.str[0:9]
        min_ = 0
        max_ = 3
        


    ##############################################
    ################### IMG ######################
    ##############################################

    if score == 'cubicWMHVOL':
        b = pd.read_csv("/home/dw/Desktop/NACC/ori_doc/mohamad07132021mri_DW.csv")[["mri_ID", 'WMHVOL']]
        b = b[(b['WMHVOL']>0) & (b['WMHVOL']<=220)]
        b[score] =  (b['WMHVOL'])**(1/3)
        min_ = 0
        max_ = 4.5


    if score == 'HIPPOVOL': #Segmented total hippocampi volume (cc)
        b = pd.read_csv("/home/dw/Desktop/NACC/ori_doc/mohamad07132021mri_DW.csv")[["mri_ID", score]]
#         print((b[score].max()))
        min_ = 0
        max_ = 8

    if score == 'cubicHIPPOVOL': #Segmented total hippocampi volume (cc)
        b = pd.read_csv("/home/dw/Desktop/NACC/ori_doc/mohamad07132021mri_DW.csv")[["mri_ID", 'HIPPOVOL']]
        b = b[(b['HIPPOVOL']>0) & (b['HIPPOVOL']<=8)]
        b[score] =  (b['HIPPOVOL'])**(1/3)
        b = b[b[score] == b[score]]
        min_ = 0
        max_ = 10

    if score == 'TEMPCOR': # Segmented total temporal lobe cortical gray matter volume (cc)
        b = pd.read_csv("/home/dw/Desktop/NACC/ori_doc/mohamad07132021mri_DW.csv")[["mri_ID", score]]
        min_ = 50
        max_ = 150

    if score == 'OCCCORT': #Segmented total occipital lobe cortical gray matter volume (cc)
        b = pd.read_csv("/home/dw/Desktop/NACC/ori_doc/mohamad07132021mri_DW.csv")[["mri_ID", score]]
        min_ = 0
        max_ = 100


    if score in ['NPAMY','NPBRAAK','NPTHAL','NPNEUR','NPINF','NPLBOD','NACCARTE','NPWMR']:
        b = b.drop_duplicates(subset = ['mri_ID'], keep = 'last')
        dfi.mri_ID = dfi.mri_ID.str[0:9].tolist()
        dfi= pd.merge(dfi, b, on = 'mri_ID', how = 'left')
        dfi= dfi[(dfi[score]>=min_) & (dfi[score]<=max_)]
    else:
        dfi= pd.merge(dfi, b, on = 'mri_ID', how = 'left')
        dfi= dfi[(dfi[score]>=min_) & (dfi[score]<=max_)]
    
    return dfi

mm=15
mi=-5
pfont = 14
ylabel_font_size = 16
xlabel_font_size = 16

def ln_plot(df, dfp, scores): # ['MMSE','cubicWMHVOL','HIPPOVOL','TRAILB','BOSTON']:
    for score in scores:
        dff = df.copy()
        df_test = get_test(dff,score)
        print(score, len(df_test))

        df_test['logit_probAD'] = np.log(df_test['probAD']/(1-df_test['probAD']) ) 
        df_test['logit_probVD'] = np.log(df_test['probVD']/(1-df_test['probVD']))
        df_test['logit_probLBD'] = np.log(df_test['probLBD']/(1-df_test['probLBD']))


        pAD,pVD,pLBD, ymin, ymax = 'logit_probAD', 'logit_probVD','logit_probLBD', -5,5



        interAD,coefAD, pvAD,r2AD = dfp[dfp['marker'] == score][['interAD','coefAD', 'pvAD_fdr','r2AD']].values[0]
        interVD,coefVD, pvVD,r2VD = dfp[dfp['marker'] == score][['interVD','coefVD', 'pvVD_fdr','r2VD']].values[0]  
        interLBD,coefLBD,pvLBD,r2LBD = dfp[dfp['marker'] == score][['interLBD','coefLBD', 'pvLBD_fdr','r2LBD']].values[0] 
        
        
        print(score, len(df_test))
        print("AD:",pvAD, round(coefAD,2))
        print("VD:",pvVD, round(coefVD,2))
        print("LBD:",pvLBD, round(coefLBD,2))
        

        fig, axs = plt.subplots(ncols = 3, nrows = 1, figsize  = (12,3))
        axs = axs.flatten()

        yy1 = pAD
        yy2 = pVD
        yy3 = pLBD

        if score == "HIPPOVOL":
            xx = "Hippocampal Volume"
        elif score == "TRAILB":
            xx = "Trail B"
        elif score == "CRAFTDVR":
            xx = "Craft Story 21 Recall"
        elif score == "LOGIMEM":
            xx = "The story units recalled"
            
        elif score == "cubicWMHVOL":
#             xx ='WMH\N{SUPERSCRIPT ONE}'
            xx = r'$\sqrt[3]{WMH Volume}$'
        else:
            xx = score
        
        
        for i, tp, yy, pv, coef, r2,inter in zip([0,1,2],['AD','VD','LBD'],[yy1,yy2,yy3],[pvAD,pvVD,pvLBD],
                                                 [coefAD,coefVD,coefLBD],[r2AD,r2VD,r2LBD],[interAD,interVD,interLBD]):
            dy = df_test[yy]
            dx = df_test[score]
            axs[i].scatter(dx, dy, s = 10, color = 'skyblue', alpha= 0.8)
            max_x = dx.max()
            min_x = dx.min()
            x = np.linspace(min_x, max_x,50)
            y = coef  * x + (inter)
            axs[i].plot(x,y, 'tomato' ,linewidth=2.5)
            if pv <= 0.05 and pv > 0.001:
                axs[i].text(.05, .86, s = r"$p^{\bf{*}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv <= 0.001 and pv > 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{**}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv < 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{***}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            elif pv > pt:
                axs[i].text(.05, .86,s = '$p_{FDR}$ > ' + str(pt), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            axs[i].text(.05,.95,s = "$n$ = " + str(len(df_test)), fontsize = pfont, 
                transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            axs[i].set_ylim([mi,mm])
            axs[i].set_xlabel(xx, fontsize = xlabel_font_size)
            axs[i].set_ylabel('$DeepSPARE$-'+r'$\bf{'+ tp+'}$', fontsize = ylabel_font_size)
            axs[i].grid(linestyle = '--')

            
        fig.tight_layout()
        


def box_plot(df, dfp, scores): # ['MMSE','cubicWMHVOL','HIPPOVOL','TRAILB','BOSTON']:
    for score in scores:
        dff = df.copy()
        df_test = get_test(dff,score)

        df_test['logit_probAD'] = np.log(df_test['probAD']/(1-df_test['probAD'])+ 0.00001) 
        df_test['logit_probVD'] = np.log(df_test['probVD']/(1-df_test['probVD'])+ 0.00001)
        df_test['logit_probLBD'] = np.log(df_test['probLBD']/(1-df_test['probLBD'])+ 0.00001)

        interAD,coefAD, pvAD,r2AD = dfp[dfp['marker'] == score][['interAD','coefAD', 'pvAD_fdr','r2AD']].values[0]
        interVD,coefVD, pvVD,r2VD = dfp[dfp['marker'] == score][['interVD','coefVD', 'pvVD_fdr','r2VD']].values[0]  
        interLBD,coefLBD,pvLBD,r2LBD = dfp[dfp['marker'] == score][['interLBD','coefLBD', 'pvLBD_fdr','r2LBD']].values[0] 
        
        
        print(score, len(df_test))
        print("AD:",pvAD, round(coefAD,2))
        print("VD:",pvVD, round(coefVD,2))
        print("LBD:",pvLBD, round(coefLBD,2))
        

        fig, axs = plt.subplots(ncols = 3, nrows = 1, figsize  = (12,3))
        axs = axs.flatten()



        if score == "NPTHAL":
            xx = "Thal Phase"
        elif score == "NPNEUR":
            xx = "CARED score"
        elif score == "NPBRAAK":
            xx = "Braak Stage"
        elif score == "NACCLEWY":
            xx = "Lewy Body Stage"
        elif score == "MEMORY":
            xx = "CDR Memory Stage"
        elif score == "NPAMY":    
            xx = "Cerebral amyloid angiopathy"
            
        elif score == 'NACCARTE':
            xx = 'Arteriolosclerosis'
        else:
            xx = score

        
        for i, tp, yy, pv, coef, r2,inter in zip([0,1,2],['AD','VD','LBD'],['logit_probAD', 'logit_probVD','logit_probLBD'],
                                                 [pvAD,pvVD,pvLBD],
                                                 [coefAD,coefVD,coefLBD],[r2AD,r2VD,r2LBD],[interAD,interVD,interLBD]):
            dy = df_test[yy]
            dx = df_test[score]
            ll = list(set(dx))
            for n, j in zip(ll, ll):
                axs[i].boxplot(df_test[df_test[score] == j][yy], positions = [n], notch=True, widths = 0.25 )
            axs[i].scatter(dx, dy, s = 10, color = 'skyblue', alpha= 0.8)
            max_x = dx.max()
            min_x = dx.min()
            x = np.linspace(min_x, max_x,50)
            y = coef  * x + (inter)
            axs[i].plot(x,y, 'tomato' ,linewidth=2.5)
            if pv <= 0.05 and pv > 0.001:
                axs[i].text(.05, .86, s = r"$p^{\bf{*}}_{FDR}$ = "+ r"${0:s}$".format(as_si(pv,2)) +\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv <= 0.001 and pv > 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{**}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv < 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{***}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            else:
                axs[i].text(.05, .86,s = '$p_{FDR}$ > ' + str(pt), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            axs[i].text(.05,.95,s = "$n$ = " + str(len(df_test)), fontsize = pfont, 
                transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            axs[i].set_ylim([mi,mm])
            axs[i].set_xlabel(xx, fontsize = xlabel_font_size)
            axs[i].set_ylabel('$DeepSPARE$-'+r'$\bf{'+ tp+'}$', fontsize = ylabel_font_size)
            axs[i].grid(linestyle = '--')
        

        for i in range(0,3):
            axs[i].grid(linestyle = '--')
            if score == "NPBRAAK":
                bars = ('0','I', 'II','III','IV', 'V', 'VI')
                y_pos = np.arange(len(bars))
                axs[i].set_xticks(y_pos, bars, fontsize='13')
            if score == "NPNEUR":
                bars = ('No','Sparse', 'Moderate','Frequent')
                y_pos = np.arange(len(bars))
                axs[i].set_xticks(y_pos, bars, fontsize='11')
            if score == "NPLBOD" or score == "NACCLEWY":
                bars = ('No','Brainstem', 'Limbic','Neocortical')
                y_pos = np.arange(len(bars))
                axs[i].set_xticks(y_pos, bars, fontsize='10.5')
            if score == "NPAMY":
                bars = ('No','Mild', 'Moderate','Severe')
                y_pos = np.arange(len(bars))
                axs[i].set_xticks(y_pos, bars, fontsize='10.5')
                
            if score == "NACCARTE":
                bars = ('No','Mild', 'Moderate','Severe')
                y_pos = np.arange(len(bars))
                axs[i].set_xticks(y_pos, bars, fontsize='10.5')
            if score == "NPTHAL":
                bars = ('0','1&2','3','4&5')
                y_pos = np.arange(len(bars))
                axs[i].set_xticks(y_pos, bars, fontsize='12')

        fig.tight_layout()
        
        

def box_plot_cohen(df,dfp, scores): 
    for score in scores:
        dff = df.copy()
        df_test = get_test(dff,score)
        print(score, len(df_test))

        df_test['logit_probAD'] = np.log(df_test['probAD']/(1-df_test['probAD']) ) 
        df_test['logit_probVD'] = np.log(df_test['probVD']/(1-df_test['probVD']))
        df_test['logit_probLBD'] = np.log(df_test['probLBD']/(1-df_test['probLBD']))


        interAD,coefAD, pvAD,r2AD = dfp[(dfp['marker'] == score)][['interAD','coefAD', 'pvAD_fdr','r2AD']].values[0]
        interVD,coefVD, pvVD,r2VD = dfp[(dfp['marker'] == score)][['interVD','coefVD', 'pvVD_fdr','r2VD']].values[0]  
        interLBD,coefLBD,pvLBD,r2LBD = dfp[ (dfp['marker'] == score)][['interLBD','coefLBD', 'pvLBD_fdr','r2LBD']].values[0] 

        syn = np.where(df_test[score]>1,1,0)
        df_test.insert(loc = 0, column = 'syn', value = syn)
#         print(df_test)
        
        if score == 'NPLBOD' or 'NACCLEWY':
            score = 'syn'
            cohenAD = cohend(df_test[df_test[score] == 1]['logit_probAD'].tolist(),df_test[df_test[score] == 0]['logit_probAD'].tolist())
            cohenVD = cohend(df_test[df_test[score] == 1]['logit_probVD'].tolist(),df_test[df_test[score] == 0]['logit_probVD'].tolist())
            cohenLBD = cohend(df_test[df_test[score] == 1]['logit_probLBD'].tolist(),df_test[df_test[score] == 0]['logit_probLBD'].tolist())


        
        print(score)
        print("AD:",pvAD, round(cohenAD,2))
        print("VD:",pvVD, round(cohenVD,2))
        print("LBD:",pvLBD, round(cohenLBD,2))
        fig, axs = plt.subplots(ncols = 3, nrows = 1, figsize  = (12,3))
        axs = axs.flatten()
     
        yy1 = pAD
        yy2 = pVD
        yy3 = pLBD
        

        if score == "syn":
            xx = "\u03B1-synuclein"
        else:
            xx = score
        
    
        for i, tp, yy, pv, cohen  in zip([0,1,2],
                                         ['AD','VD','LBD'],
                                         ['logit_probAD', 'logit_probVD','logit_probLBD'],
                                         [pvAD,pvVD,pvLBD],
                                         [cohenAD,cohenVD,cohenLBD]):
            dy = df_test[yy]
            dx = df_test[score]
            ll = list(set(dx))
            for n, j in zip(ll, ll):
                axs[i].boxplot(df_test[df_test[score] == j][yy], positions = [n], notch=True, widths = 0.25 )
            axs[i].scatter(dx, dy, s = 10, color = 'skyblue', alpha= 0.8)

            if pv <= 0.05 and pv > 0.001:
                axs[i].text(.05, .86, s = r"$p^{\bf{*}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$Cohen'd$ = " + str(np.round(cohen,3)), fontsize = pfont,  
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv <= 0.001 and pv > 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{**}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$Cohen'd$ = " + str(np.round(cohen,3)), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv < 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{***}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$Cohen'd$ = " + str(np.round(cohen,3)), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            elif pv > pt:
                axs[i].text(.05, .86,s = '$p_{FDR}$ > ' + str(pt), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            axs[i].text(.05,.95,s = "$n$ = " + str(len(df_test)), fontsize = pfont, 
                transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            axs[i].set_ylim([mi,mm])
            axs[i].set_xlabel(xx, fontsize = xlabel_font_size)
            axs[i].set_ylabel('$DeepSPARE$-'+r'$\bf{'+ tp+'}$', fontsize = ylabel_font_size)
            axs[i].grid(linestyle = '--')
        for i in range(0,3):
            if score == "syn":
                bars = ('negtive', 'positive')
                y_pos = np.arange(len(bars))
                axs[i].set_xticks(y_pos, bars, fontsize='13')
        
        fig.tight_layout()
        
def as_si(x, ndp):
    s = '{x:0.{ndp:d}e}'.format(x=x, ndp=ndp)
    m, e = s.split('e')
    return  r'{m:s}\times 10^{{{e:d}}}'.format(m=m, e=int(e))       

# FDR correction

In [ ]:
l = []
markers = ['MMSE','TRAILB','LOGIMEM','HIPPOVOL','cubicWMHVOL','NACCARTE','NPAMY','NPTHAL','NPNEUR','NPBRAAK','NACCLEWY']

for score in markers:
    dff = df.copy()
    df_test = get_test(dff,score)
#     print(score, len(df_test))

    df_test['logit_probAD'] = np.log(df_test['probAD']/(1-df_test['probAD']) ) 
    df_test['logit_probVD'] = np.log(df_test['probVD']/(1-df_test['probVD']))
    df_test['logit_probLBD'] = np.log(df_test['probLBD']/(1-df_test['probLBD']))

    pAD,pVD,pLBD, ymin, ymax = 'logit_probAD', 'logit_probVD','logit_probLBD', -5,5

    interAD,coefAD, pvAD = ln("AD" , df_test[score], df_test[[pAD]])
    interVD,coefVD, pvVD = ln("VD" , df_test[score], df_test[[pVD]])  
    interLBD,coefLBD,pvLBD = ln("LBD" , df_test[score], df_test[[pLBD]]) 
    
    
    r2AD = get_r2(df_test[[score]], df_test[[pAD]])
    r2VD = get_r2(df_test[[score]], df_test[[pVD]])
    r2LBD = get_r2(df_test[[score]], df_test[[pLBD]])
    
    if score == 'NACCLEWY' or score =='NPLBOD':
        syn = np.where(df_test[score]>1,1,0)
        df_test.insert(loc = 0, column = 'syn', value = syn)
        score = 'syn'
        pvAD = ttest_ind(df_test[df_test[score]==1][pAD].tolist(), df_test[df_test[score]==0][pAD].tolist())[1]
        pvVD = ttest_ind(df_test[df_test[score]==1][pVD].tolist(), df_test[df_test[score]==0][pVD].tolist())[1]
        pvLBD = ttest_ind(df_test[df_test[score]==1][pLBD].tolist(), df_test[df_test[score]==0][pLBD].tolist())[1]



    l.append([interAD,coefAD, pvAD,interVD,coefVD, pvVD,interLBD,coefLBD,pvLBD,r2AD,r2VD,r2LBD])
    
    
dfp = pd.DataFrame(l,columns = ['interAD','coefAD', 'pvAD','interVD','coefVD', 'pvVD','interLBD','coefLBD','pvLBD','r2AD','r2VD','r2LBD'])
dfp.insert(loc = 0, column = 'marker', value = markers)
pa = dfp.pvAD.tolist() +dfp.pvVD.tolist()+dfp.pvLBD.tolist()
pa

fdr = statsmodels.stats.multitest.fdrcorrection(pa, alpha=0.05, is_sorted=False)[1]
fdr

dfp.insert(loc = 1, column = 'pvAD_fdr', value =fdr[0:len(markers)])
dfp.insert(loc = 1, column = 'pvVD_fdr', value =fdr[len(markers):len(markers)*2])
dfp.insert(loc = 1, column = 'pvLBD_fdr', value =fdr[len(markers)*2:len(markers)*3])
dfp

In [ ]:
ln_plot(df,dfp,['MMSE','TRAILB','LOGIMEM'])

In [ ]:
ln_plot(df,dfp,['HIPPOVOL','cubicWMHVOL'])

In [ ]:
box_plot(df,dfp,['NACCARTE','NPAMY','NPTHAL','NPNEUR','NPBRAAK','NACCLEWY']) #,'NPNEUR','NPBRAAK','NACCLEWY'